In [2]:
from numpy import linalg
import numpy as np
import pandas as pd
from sklearn.preprocessing import normalize, scale, Normalizer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

In [3]:
df = pd.read_csv('spotify_songs_final_filtered.csv')
df.shape

(455285, 24)

In [4]:
df.columns

Index(['track_id', 'artist', 'song_name', 'popularity', 'album', 'isrc',
       'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms',
       'time_signature'],
      dtype='object')

In [5]:
df.iloc[:2][:1]

,track_id,artist,song_name,popularity,album,isrc,danceability,energy,key,loudness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,4YYCyU5iK3l71ksslONDAd,''Sugar Boy'' Crawford,Jock-A-Mo,4,"Louisiana Jazz, Rhythm & Blues - From the Swam...",USMC15350776,0.54,0.587,7,-7.982,...,0.245,0.948,168.823,audio_features,4YYCyU5iK3l71ksslONDAd,spotify:track:4YYCyU5iK3l71ksslONDAd,https://api.spotify.com/v1/tracks/4YYCyU5iK3l7...,https://api.spotify.com/v1/audio-analysis/4YYC...,148800,4


In [6]:
#preprocess
feat = df[['popularity', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']].copy()
feat = normalize(feat, norm='l2')
feat.shape

(455285, 12)

In [9]:
#weights
cats = ['popularity', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']
user_inputs = [1,2,3,4,5,1,2,3,4,5,1,2]
#user_inputs = [1,1,1,1,1,1,1,1,1,1,1,1]
#user_inputs = 1/(np.array(user_inputs)**(2)) not the relationship we want
#user_inputs = np.array(user_inputs)
weights = dict(zip(cats, user_inputs))
weights

{'acousticness': 3,
 'danceability': 2,
 'energy': 3,
 'instrumentalness': 4,
 'key': 4,
 'liveness': 5,
 'loudness': 5,
 'mode': 1,
 'popularity': 1,
 'speechiness': 2,
 'tempo': 2,
 'valence': 1}

In [10]:
feat = pd.DataFrame(feat, columns=weights)
feat.head()

,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0.023639,0.003191,0.003469,0.041368,-0.047171,0.005910,0.000351,0.004172,0.000000e+00,0.001448,0.005602,0.997696
1,0.182105,0.004811,0.004522,0.053114,-0.059184,0.007588,0.000414,0.005698,2.837798e-08,0.000511,0.006837,0.979966
2,0.072938,0.003005,0.003793,0.058351,-0.050663,0.007294,0.000607,0.006389,0.000000e+00,0.000579,0.002786,0.994275
3,0.060733,0.003894,0.004517,0.068324,-0.042801,0.007592,0.000686,0.005762,4.995264e-06,0.000758,0.002961,0.994824
4,0.073607,0.004837,0.005174,0.094638,-0.082766,0.010515,0.001157,0.008297,4.626757e-07,0.001514,0.001840,0.989211


In [29]:
#scaled
test = user_inputs * feat
test.head()

,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0.023639,0.006382,0.010407,0.165472,-0.235857,0.005910,0.000702,0.012517,0.000000e+00,0.007239,0.005602,1.995391
1,0.182105,0.009621,0.013567,0.212455,-0.295920,0.007588,0.000829,0.017095,1.135119e-07,0.002553,0.006837,1.959932
2,0.072938,0.006010,0.011378,0.233403,-0.253315,0.007294,0.001214,0.019168,0.000000e+00,0.002896,0.002786,1.988550
3,0.060733,0.007789,0.013551,0.273297,-0.214007,0.007592,0.001371,0.017286,1.998105e-05,0.003792,0.002961,1.989648
4,0.073607,0.009674,0.015521,0.378553,-0.413832,0.010515,0.002313,0.024890,1.850703e-06,0.007571,0.001840,1.978422


In [15]:
weights

{'acousticness': 3,
 'danceability': 2,
 'energy': 3,
 'instrumentalness': 4,
 'key': 4,
 'liveness': 5,
 'loudness': 5,
 'mode': 1,
 'popularity': 1,
 'speechiness': 2,
 'tempo': 2,
 'valence': 1}

In [16]:
test.iloc[rec]

,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,scores
0,0.023639,0.006382,0.010407,0.165472,-0.235857,0.005910,0.000702,0.012517,0.000000e+00,0.007239,0.005602,1.995391,0.000000
384398,0.024156,0.006377,0.008986,0.169089,-0.234852,0.006039,0.000377,0.011486,1.792340e-05,0.009964,0.004952,1.995319,0.005041
271383,0.027115,0.005545,0.009680,0.162687,-0.234914,0.006779,0.000485,0.008663,1.819386e-03,0.009422,0.001607,1.995319,0.007852
189583,0.029701,0.003220,0.008322,0.166327,-0.232235,0.005940,0.000396,0.012920,0.000000e+00,0.003653,0.001830,1.995158,0.009610
398325,0.024926,0.008242,0.019218,0.166174,-0.233142,0.008309,0.000553,0.015305,8.375174e-05,0.005733,0.005750,1.995286,0.010318
187237,0.016690,0.008178,0.016923,0.166898,-0.237078,0.008345,0.000596,0.015021,2.173010e-05,0.005090,0.004356,1.995564,0.010766
87996,0.030288,0.006990,0.010867,0.169612,-0.234610,0.006058,0.000390,0.005052,2.544178e-06,0.006663,0.004798,1.994987,0.010976
191835,0.017418,0.005016,0.014091,0.162566,-0.239001,0.000000,0.000446,0.008709,5.945259e-04,0.005864,0.003820,1.995703,0.011291
280240,0.023009,0.004878,0.015548,0.161063,-0.242429,0.005752,0.000381,0.008922,3.520384e-05,0.012339,0.004642,1.995391,0.011476
240503,0.017147,0.004572,0.009482,0.160036,-0.241626,0.005716,0.000528,0.010802,1.634654e-07,0.003115,0.003024,1.995695,0.011656


### PCA, KMeans

In [47]:
#test is the normalized df
kmeans = KMeans(n_clusters = 20, random_state=0, algorithm='full').fit(test)

In [48]:
#compress to only two main components
pca = PCA(n_components=2)
pca_df = pca.fit(test.T)
pca_df = pd.DataFrame(pca_df.components_.T, columns=['component_1', 'component_2'])

In [49]:
#add cluster assignments
pca_df['cluster']=kmeans.labels_
pca_df.cluster.value_counts()

0     52245
19    47724
11    45163
2     41830
10    35580
5     31531
13    30439
4     28664
16    26821
6     21032
9     19328
12    17485
8     16609
1     12301
7      9831
17     7749
18     7209
14     3030
15      444
3       270
Name: cluster, dtype: int64

In [50]:
#data to share with group
#pca_df.to_csv('full_data_set_pca.csv', index=False)

In [54]:
seed_index = 25
test['cluster']=kmeans.labels_
#cluster = test.iloc[seed_index]['cluster']
#cluster_df = test.loc[test['cluster']==cluster].copy()
test['scores']= linalg.norm(test.iloc[seed_index] - test, axis=1)

,scores,cluster
0,12.015046,0
1,2.037345,10
2,12.011482,0
3,12.013923,0
4,4.021879,16


In [60]:
rec = test.sort_values("scores").index[:25]
#df.iloc[rec][['song_name', 'artist']+cats]
test.iloc[rec][['scores', 'cluster']]

,scores,cluster
25,0.000000,12
279924,0.015457,12
9279,0.018719,12
3658,0.020714,12
231682,0.020987,12
112068,0.022402,12
302812,0.022568,12
116374,0.023908,12
294963,0.024728,12
323711,0.025414,12
